# Optimization


In [1]:
from functools import partial
from temfpy.optimization import carlberg

from scipy import optimize as opt
import matplotlib.pyplot as plt
from scipy.stats import norm
import seaborn as sns
import scipy as sp
import pandas as pd
import numpy as np

from optimization_problems import get_test_function_gradient
from optimization_problems import get_parameterization
from optimization_problems import get_test_function
from optimization_auxiliary import process_results
from optimization_auxiliary import get_bounds
from optimization_plots import plot_contour
from optimization_plots import plot_surf
from optimization_plots import plot_optima_example
from optimization_plots import plot_true_observed_example

ModuleNotFoundError: No module named 'temfpy'

## Outline

1. Setup
2. Algorithms
3. Gradient-based methods
4. Derivative-free methods
5. Benchmarking exercise
6. Special cases



## Setup

In the finite-dimensional unconstrained optimization problem, one is given a function $f : R^n \mapsto R$ and asked to find an $x^\ast$ such that $f (x^\ast) \geq f(x)$ for all $x$. We call $f$ the objective function and $x^\ast$ , if it exists, the global minimum of $f$. We focus on minimum - to solve a minimization problem, simply minimize the negative of the objective.

We say that $x^\ast \in R^n$ is a ...

* strict global minimum of $f$ if $f(x^\ast) > f (x)$ for all $x\neq x^\ast$.
* weak local minimum of $f$ if $f(x^\ast) \geq f(x)$ for all $x$ in some neighborhood of $x^\ast$.
* strict local minimum of $f$ if $f(x^\ast) > f(x)$ for all $x\neq x^\ast$ in some neighborhood of $x^\ast$.

In [2]:
plot_optima_example()

NameError: name 'plot_optima_example' is not defined

Let $f: R^n \mapsto R$ be twice continuously differentiable.

* **First Order Necessary Conditions:** If $x^\ast$ is a local minimum of $f$, then $f^{\prime}(x^\ast) = 0$. 
* **Second Order Necessary Condition:** If $x^\ast$ is a local minimum of $f$, then $f^{\prime\prime}(x^*)$ is negative semidefinite. 

We say $x$ is a critical point of $f$ if it satisfies the first-order necessary condition.

* **Sufficient Condition:** If $f^\prime (x^\ast) = 0$ and $f^{\prime\prime}(x^\ast)$ is negative definite, then $x^\ast$ is a strict local minimum of $f$.
* **Local-Global Theorem:** If $f$ is concave, and $x^\ast$ is a local minimum of $f$, then $x^\ast$ is a global minimum of $f$.

**Key problem attributes**

* Convexity: convex vs. non-convex
* Optimization-variable type: continuous vs. discrete
* Constraints: unconstraint vs. constraint
* Number of optimization variables: low-dimensional vs. high-dimensional

These attributes dictate:

* ability to find solution
* problem complexity and computing time
* appropriate methods
* relevant software

$\Rightarrow$ Always begin by categorizing your problem



Optimization problems are ubiquitous in economics:

* Government maximizes social welfare
* Competitive equilibrium maximizes total surplus
* Ordinary least squares estimator minimizes sum of squares
* Maximum likelihood estimator maximizes likelihood function


## Algorithms


We are mostly blind to the function we are trying to minimize and can only compute the function at a limited number of points. Each evaluation is computationally expensive.

In [ ]:
plot_true_observed_example()

**Goals**

* reasonable memory requirements
* low failure rate, convergence conditions are met
* convergence in a few iterations with low cost for each iteration

**Catergorization**

* gradient-based vs. derivative-free
* global vs. local


### _Question_  

* How to compute derivatives?


## Gradient-based methods

**Benefits**

* efficient for many variables
* well-suited for smooth objective and constraint functions

**Drawbacks**

* requires computing the gradient, potentially challenging and time-consuming
* convergence is only local
* not-well suited for noisy functions, derivative information flawed

Second derivative are also very useful, but ...

* Hessians are $n\times n$, so expensive to construct and store
* often only approximated using quasi-Newton methods

### _Questions_

1. How to use gradient-based algorithms to find a global optimum?
2. Any ideas on how to reduce the memory requirements for a large Hessian?

<img src="material/fig-gradient-based-algorithm.png" width=500 height=500 />


<img src="material/fig-gradient-based-overview.png" width=500 height=500 />


There is two different classes of gradient-based algorithms.

* Line-search methods
    * compute $p_k$ be a descent direction
    * compute $a_k$ to produce a sufficient decrease in the objective function

Let's see [here](https://github.com/scipy/scipy/blob/8e30f7797bd1ee442f4f1a25172e4402521c1e16/scipy/optimize/optimize.py#L1484) for how such a line search looks like in practice for the [Newton-CG](https://en.wikipedia.org/wiki/Nonlinear_conjugate_gradient_method) algorithm.

* Trust-region methods
    * determine a maximum allowable step length (trust-region radius) $\delta k$
    * compute step $k$ with $||p_k|| \leq \Delta$ using a model $m(p) \approx f(x_k + p)$

As an example implementation, see [here](https://github.com/scipy/scipy/blob/46b359bfb54072dae61882731bb2e766e38ba393/scipy/optimize/_trustregion.py#L100) for the `scipy.optimize_trustregion.py` implementation.



## Derivative-Free Methods

**Benefits**

* often better at finding a global minimum if function not convex
* robust with respect to noise in criterion function
* amenable to parallelization


**Drawbacks**

* extremely slow convergence for high-dimensional problems

There are two different classes of derivative-free algorithms.

* heuristic, inspired by nature
    * basin-hopping
    * evolutionary algorithms

* direct search
    * directional
    * simplicial

### Test function


$$
f(x) = \tfrac{1}{2}\sum_{i=1}^n a_i\cdot (x_i-1)^2+ b\cdot \left[ n-\sum_{i=1}^n\cos(2\pi(x_i-1))\right],
$$

where $a_i$ and $b$ provide the parameterization of the function.


### _Exercises_

1. Implement this test function.
2. Visualize the shape of our test function for the one-dimensional case.
3. What is the role of the parameters $a_1$ and $b$?
4. What is the functions global minimum? 
   

In [ ]:
??get_test_function

In [ ]:
??get_parameterization

We want to be able to use our test function for different configurations of the challenges introduced by noise and ill-conditioning.

In [ ]:
add_noise, add_illco, x0 = False, False, [4.5, -1.5]


def get_problem(dimension, add_noise, add_illco, seed=123):
    np.random.seed(seed)
    a, b = get_parameterization(dimension, add_noise, add_illco)
    get_test_function_p = partial(get_test_function, a=a, b=b)
    get_test_function_gradient_p = partial(get_test_function_gradient, a=a, b=b)
    return get_test_function_p, get_test_function_gradient_p


dimension = len(x0)
opt_test_function, opt_test_gradient = get_problem(dimension, add_noise, add_illco)
np.testing.assert_equal(opt_test_function([1, 1]), 0.0)

Let's see how the surface and contour plots look like under different scenarios.

In [ ]:
opt_test_function, _ = get_problem(dimension, add_noise, add_illco)
plot_surf(opt_test_function)

### _Question_

* How is the global minimum affected by the addition of noise and ill-conditioning?


## Benchmarking exercise


Let's get our problem setting and initialize a container for our results. We will use the convenient interface to [scipy.optimize.minimize](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html#scipy.optimize.minimize). Its documentation also points you to research papers and textbooks where the details of the algorithms are discussed in more detail. We need to invest a little in the design of our setup first, but then we can run the benchmarking exercise with ease and even adding additional optimization algorithms is straightforward.

In [ ]:
ALGORITHMS = ["CG", "Newton-CG", "Nelder-Mead", "Diff-Evol"]
add_noise, add_illco, dimension = False, False, 2

In [ ]:
x0 = [4.5, -1.5]
opt_test_function, opt_test_gradient = get_problem(dimension, add_noise, add_illco)
df = pd.DataFrame(columns=["Iteration", "Distance"], index=ALGORITHMS)
df.index.name = "Method"

Let's fix what will stay unchanged throughout.

In [ ]:
call_optimizer = partial(
    opt.minimize,
    fun=opt_test_function,
    x0=x0,
    jac=opt_test_gradient,
    options={"disp": True, "return_all": True, "maxiter": 100000},
)

We prepared some functions to process results from the optimizer calls.

In [ ]:
??process_results

### Conjugate gradient


In [ ]:
method = "CG"
res = call_optimizer(method=method)
df = process_results(df, method, res)
plot_contour(opt_test_function, res["allvecs"], method)

###  Newton-CG

In [ ]:
method = "Newton-CG"
res = call_optimizer(method=method)
df = process_results(df, method, res)
plot_contour(opt_test_function, res["allvecs"], method)

### Nelder Mead

In [ ]:
method = "Nelder-Mead"
res = call_optimizer(method=method)
df = process_results(df, method, res)
plot_contour(opt_test_function, res["allvecs"], method)

### Differential evolution

In [ ]:
??get_bounds

In [ ]:
method = "Diff-Evol"
res = opt.differential_evolution(opt_test_function, get_bounds(dimension))
plot_contour(opt_test_function, res["x"], method)
df = process_results(df, method, res)

### Summary

In [ ]:
_ = sns.barplot(x="Method", y="Iteration", data=df.reset_index())

In [ ]:
_ = sns.barplot(x="Method", y="Distance", data=df.reset_index())

### Speeding up test function

We want to increase the dimensionality of our optimization problem going forward. Even in this easy setting, it is worth to re-write our objective function using `numpy` to ensure its speedy execution. A faster version is already available as part of the Python package temfpy. Below, we compare our test function to the [temfpy version](https://temfpy.readthedocs.io/en/latest/optimization.html#temfpy.optimization.carlberg) and assess their performance in regard to speed.

In [ ]:
??get_test_function

In [ ]:
??carlberg

It is very easy to introduce errors when speeding up your code as usually you face a trade-off between readability and performance. However, setting up a simple testing harness that simply compares the results between the slow, but readable, implementation and the fast one for numerous random test problems. For more automated, but random, testing see [Hypothesis](https://hypothesis.readthedocs.io/en/latest/).

In [ ]:
def get_speed_test_problem():
    add_illco, add_noise = np.random.choice([True, False], size=2)
    dimension = np.random.randint(2, 100)

    a, b = get_parameterization(dimension, add_noise, add_illco)
    x0 = np.random.uniform(size=dimension)
    return x0, a, b

Now we are ready to put our fears at ease.

In [ ]:
for _ in range(1000):
    args = get_speed_test_problem()
    stats = get_test_function(*args), carlberg(*args)
    np.testing.assert_almost_equal(*stats)

Let's see whether this was worth the effort for a small and a large problem using the `%timeit` magic function.

In [ ]:
dimension, add_noise, add_illco = 100, True, True
x0 = np.random.uniform(size=dimension)
a, b = get_parameterization(dimension, add_noise, add_illco)

In [ ]:
%timeit carlberg(x0, a, b)

In [ ]:
%timeit get_test_function(x0, a, b)

In this particular setting, there is no need to increase the performance even further. However, as a next step, check out [numba](https://numba.pydata.org/), for even more flexibility in speeding up your code.

### _Exercises_

1. Repeat the exercise in the case of noise in the criterion function and try to summarize your findings. 
2. What additional problems arise as the dimensionality of the problem for a 100-dimensional problem? Make sure to use the fast implementation of the test function. 


## Special cases
        

Nonlinear least squares and maximum likelihood estimation have special structure that can be exploited to improve the approximation of the inverse Hessian.


### Nonlinear least squares

We will estimate the following nonlinear consumption function using data from Greene's textbook:

$$
\begin{align*}
C = \alpha  + \beta \times Y^\gamma + \epsilon 
\end{align*}
$$

which is estimated with quarterly data on real consumption and disposable income for the U.S. economy from 1950 to 2000.

In [ ]:
df = pd.read_pickle("material/data-consumption-function.pkl")
df.head()

Let's confirm the basic relationship to get an idea of what to expect for the estimated parameters.

In [ ]:
fig, ax = plt.subplots()
x = df.index.get_level_values("Year")

for name in ["realgdp", "realcons"]:
    y = df[name]
    ax.plot(x, y, label=name)

ax.set_xlabel("Year")
ax.set_ylabel("Value")
ax.legend()

Now we set up the criterion function such that it fits the requirements.

In [ ]:
consumption = df["realcons"].values
income = df["realgdp"].values


def ssr(x, consumption, income):
    alpha, beta, gamma = x
    residuals = consumption - alpha - beta * income ** gamma
    return residuals


ssr_partial = partial(ssr, consumption=consumption, income=income)
rslt = sp.optimize.least_squares(ssr_partial, [0, 0, 1])["x"]

### _Exercise_

* Evaluate the fit of the model.

### Maximum likelihood estimation

Greene (2012) considers the following binary choice model.

$$
\begin{align*}
P[Grade = 1] = F(\beta_0 + \beta_1 GPA + \beta_2 TUCE + \beta_3 PSI)
\end{align*}
$$

where $F$ the cumulative distribution function for either the normal distribution (Probit) or the logistic distribution (Logit). 

In [ ]:
df = pd.read_pickle("material/data-graduation-prediction.pkl")
df.head()

In [ ]:
def probit_model(beta, y, x):
    F = norm.cdf(x @ beta)
    fval = (y * np.log(F) + (1 - y) * np.log(1 - F)).sum()
    return -fval

In [ ]:
x, y = df[["INTERCEPT", "GPA", "TUCE", "PSI"]], df["GRADE"]
rslt = opt.minimize(probit_model, [0.0] * 4, args=(y, x))

### _Exercise_

* Amend the code so that you can simply switch between estimating a Probit or Logit model.


## Resources


* **Kevin T. Carlberg**: https://kevintcarlberg.net

### Software 

* **Ipopt**: https://coin-or.github.io/Ipopt

* **SNOPT (Sparse Nonlinear OPTimizer)**: https://ccom.ucsd.edu/~optimizers/solvers/snopt

* **Gurobi** https://www.gurobi.com

* **IBM CPLEX Optimizer** https://www.ibm.com/analytics/cplex-optimizer

### Books

* Nocedal, J., & Wright, S. (2006). [*Numerical optimization*](https://www.amazon.de/-/en/Jorge-Nocedal/dp/0387303030/ref=sr_1_1?dchild=1&keywords=nocedal+wright&qid=1605168584&sr=8-1) . Springer Science & Business Media.

* Boyd, S., Boyd, S. P., & Vandenberghe, L. (2004). [*Convex optimization*](https://www.amazon.de/-/en/Stephen-Boyd/dp/0521833787/ref=sr_1_3?dchild=1&keywords=optimization&qid=1605168744&sr=8-3). Cambridge university press.

* Kochenderfer, M. J., & Wheeler, T. A. (2019). [*Algorithms for optimization*](https://www.amazon.de/-/en/Mykel-J-Kochenderfer/dp/0262039427/ref=sr_1_1?dchild=1&keywords=optimization&qid=1605168758&sr=8-1). Mit Press.

* Fletcher, R. (2000). [*Practical methods of optimization (2nd edn)*](https://www.amazon.com/gp/product/0471494631/ref=ox_sc_act_title_1?ie=UTF8&smid=ATVPDKIKX0DER#reader_0471494631). Wiley.

* Nesterov, Y. (2018). [*Lectures on convex optimization*](https://www.springer.com/gp/book/9783319915777#aboutBook). Springer Nature Switzerland.

### Research

* Moré, J. J., & Wild, S. M. (2009). [Benchmarking derivative-free optimization algorithms](https://epubs.siam.org/doi/abs/10.1137/080724083?journalCode=sjope8). *SIAM Journal on Optimization*, 20(1), 172-191.

* Beiranvand, V., Hare, W., & Lucet, Y. (2017). [Best practices for comparing optimization algorithms](https://link.springer.com/article/10.1007/s11081-017-9366-1). *Optimization and Engineering*, 18, 815–848.

* Bartz-Beielstein, T., et al. (2020). [Benchmarking in optimization: Best practice and open issues](https://link.springer.com/article/10.1007/s11081-017-9366-1). *arXiv preprint arXiv:2007.03488.*.
